In [1]:
import pandas as pd
import numpy as np 
import json
import os
from pprint import pprint
from konlpy.tag import Okt

### 데이터는 테스트용으로 전에 세정력만 전처리한 데이터를 썼습니다. 샘플이니깐 원하는 데이터셋  가져다 쓰면 되요 

In [2]:
data1 = pd.read_excel('샴푸샴푸세정력.xlsx')

In [3]:
data1.head()

,Unnamed: 0,Unnamed: 0.1,product,review,세정력,cleansing,세정gram,뽀득gram,세정pos,뽀득pos,pos
0,0,0,허벌에센스 핑크로즈 샴푸 490ml,기존에 상품대비 가격이 부담이었는데 할인율이 높아져서 만족하면서 사용하고 있어요...,아주 만족해요,0,NaN,NaN,0,0,0
1,1,1,허벌에센스 핑크로즈 샴푸 490ml,뽀득뽀득해요 성분 좋다는 샴푸 쓰다가 거품도 너무 안나고 세정력이 떨어져서 바꿔봤는...,보통이에요,1,"['세정력이', '떨어져서', '바꿔봤는데', '뽀득뽀득', '씻겨서']","['뽀득뽀득해요', '성분', '좋다는', '샴푸', '쓰다가']",0,1,1
2,2,2,허벌에센스 핑크로즈 샴푸 490ml,매번쓰던거입니다 거품도풍넝하고맘에아두듭니다,아주 만족해요,0,NaN,NaN,0,0,0
3,3,3,허벌에센스 핑크로즈 샴푸 490ml,노란색 사용하다가 핑크색이 향이 더 좋을것 같아서 구매했어요,보통이에요,0,NaN,NaN,0,0,0
4,4,4,허벌에센스 핑크로즈 샴푸 490ml,비누와 같은 세정력 세장력은 좋은 것 같은데 샴푸 후 머릿결이 조금 뻣뻣한 느낌이 ...,보통이에요,1,"['세정력', '세장력은', '좋은', '것', '같은데']",NaN,1,0,1


In [4]:
data2 = data1.loc[:,['review','pos']]

In [5]:
data2.shape

(19896, 2)

In [6]:
data2.head()

,review,pos
0,기존에 상품대비 가격이 부담이었는데 할인율이 높아져서 만족하면서 사용하고 있어요...,0
1,뽀득뽀득해요 성분 좋다는 샴푸 쓰다가 거품도 너무 안나고 세정력이 떨어져서 바꿔봤는...,1
2,매번쓰던거입니다 거품도풍넝하고맘에아두듭니다,0
3,노란색 사용하다가 핑크색이 향이 더 좋을것 같아서 구매했어요,0
4,비누와 같은 세정력 세장력은 좋은 것 같은데 샴푸 후 머릿결이 조금 뻣뻣한 느낌이 ...,1


In [7]:
data2['pos'].isnull().sum()

0

In [8]:
data2.dropna(subset=['review'],inplace=True)

In [9]:
from sklearn.model_selection import train_test_split

class_df = data2['pos']
feature_df = data2.drop(['pos'], axis=1, inplace=False)

#df_train, df_test = train_test_split(data2, test_size=0.3, random_state=0)
X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape
#df_train.shape, df_test.shape

((13917, 1), (5965, 1))

## 1. 머신러닝으로 감성분석 분류기 테스트_CountVectorizer

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

/home/daehynk/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


예측 정확도는 0.9915, ROC-AUC는 0.9942


In [11]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

/home/daehynk/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


예측 정확도는 0.9920, ROC-AUC는 0.9929


## 2. 딥러닝으로 감성 분석 코드 

In [14]:
from konlpy.tag import Mecab

mecab = Mecab()

In [16]:
import json
import os
from pprint import pprint

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in mecab.pos(doc)]

if os.path.isfile('train_docs.json'):
    with open('train_docs.json') as f:
        train_docs = json.load(f)
    with open('test_docs.json') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row),label ) for row,label in zip(X_train['review'],y_train)]
    test_docs = [(tokenize(row),label ) for row,label in zip(X_test['review'],y_test)]
    # JSON 파일로 저장
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

# 예쁘게(?) 출력하기 위해서 pprint 라이브러리 사용
pprint(train_docs[0])

(['최고/NNG',
  '입니다/VCP+EF',
  '역시/MAJ',
  '바티스트/NNG',
  '만큼/JKB',
  '가/JKS',
  '성비/NNG',
  '좋/VA',
  '고/EC',
  '좋/VA',
  '은/ETM',
  '드라이/NNG',
  '샴푸/NNG',
  '가/JKS',
  '없/VA',
  '는/ETM',
  '것/NNB',
  '같/VA',
  '아요/EF',
  '몇/MM',
  '통/NNBC',
  '째/XSN',
  '구매/NNG',
  '인지/VCP+EC',
  '기억/NNG',
  '도/JX',
  '안/MAG',
  '나/VV',
  '네여/EF',
  '머리/NNG',
  '가/JKS',
  '길/VA',
  '어서/EC',
  '매일/MAG',
  '감기/NNG',
  '가/JKS',
  '든/VV+ETM',
  '데/NNB',
  '늘/MAG',
  '쟁여/VV+EC',
  '뒀/VX+EP',
  '다가/EC',
  '유용/NNG',
  '하/XSV',
  '게/EC',
  '사용/NNG',
  '합니다/XSV+EC'],
 0)


In [21]:
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

360867


In [22]:
import nltk
text = nltk.Text(tokens,name='세정력')
print(text)

<Text: 세정력>


In [23]:
# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))            

# 출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))

360867
10299
[('좋/VA', 12868),
 ('고/EC', 10269),
 ('이/JKS', 6630),
 ('도/JX', 6586),
 ('에/JKB', 5825),
 ('샴푸/NNG', 4438),
 ('는/ETM', 4308),
 ('어요/EF', 4255),
 ('아요/EF', 4125),
 ('가/JKS', 3970)]


In [24]:
selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [25]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [26]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
13917/13917 [==============================] - 2s 159us/sample - loss: 0.2976 - binary_accuracy: 0.9544
Epoch 2/10
13917/13917 [==============================] - 2s 123us/sample - loss: 0.1485 - binary_accuracy: 0.9846
Epoch 3/10
13917/13917 [==============================] - 2s 145us/sample - loss: 0.0724 - binary_accuracy: 0.9854
Epoch 4/10
13917/13917 [==============================] - 2s 134us/sample - loss: 0.0336 - binary_accuracy: 0.9898
Epoch 5/10
13917/13917 [==============================] - 2s 128us/sample - loss: 0.0176 - binary_accuracy: 0.9943
Epoch 6/10
13917/13917 [==============================] - 2s 126us/sample - loss: 0.0096 - binary_accuracy: 0.9972
Epoch 7/10
13917/13917 [==============================] - 2s 137us/sample - loss: 0.0057 - binary_

In [27]:
results

[0.13028755275269324, 0.9875943]

In [28]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;\n".format(review, (1 - score) * 100))

In [29]:
predict_pos_neg("뽀득하고 세정이 잘 되는거 같아요.")


[뽀득하고 세정이 잘 되는거 같아요.]는 80.92% 확률로 긍정 리뷰이지 않을까 추측해봅니다.^^



In [30]:
predict_pos_neg("세정력 좋아요 향은 별로네요 두피가 시원해요")

[세정력 좋아요 향은 별로네요 두피가 시원해요]는 99.91% 확률로 부정 리뷰이지 않을까 추측해봅니다.^^;

